In [4]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')
shanghai = pd.read_csv('D:\\FILE\\백령도대기질.csv')  # 상하이 데이터 추가

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')
shanghai['date'] = pd.to_datetime(shanghai['date'], errors='coerce')  # 상하이 데이터

# 2023년 데이터를 필터링
beijing_past = beijing[beijing['date'] < '2024-01-01']
seoul_past = seoul[seoul['date'] < '2024-01-01']
shanghai_past = shanghai[shanghai['date'] < '2024-01-01']  # 상하이 데이터

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)
shanghai_numeric = preprocess(shanghai_past)  # 상하이 데이터

# 서울과 베이징, 상하이의 데이터 병합
corr_beijing_seoul_shanghai = pd.merge(beijing_numeric, seoul_numeric, 
                                       left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))
corr_beijing_seoul_shanghai = pd.merge(corr_beijing_seoul_shanghai, shanghai_numeric, 
                                       left_index=True, right_index=True, suffixes=('', '_shanghai'))
shanghai

,date,pm25,pm10,o3,no2,so2,co
0,2024-07-01,42,17,50,1,2,3
1,2024-07-02,34,24,67,1,2,4
2,2024-07-04,61,10,57,,2,4
3,2024-07-05,17,18,65,1,2,3
4,2024-07-06,41,16,60,2,2,3
...,...,...,...,...,...,...,...
3595,2014-03-27,,,60,2,2,4
3596,2014-03-28,,,52,3,,4
3597,2014-03-29,,,70,5,6,5
3598,2014-03-30,,,75,3,9,4


In [37]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# PM2.5 예측 모델
X_pm25 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm25 = corr_beijing_seoul[' pm25_seoul']

# PM10 예측 모델
X_pm10 = corr_beijing_seoul[[' pm25_beijing', ' pm10_beijing']]
y_pm10 = corr_beijing_seoul[' pm10_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# PM2.5, PM10 데이터 준비
X_pm25, y_pm25 = prepare_data(X_pm25, y_pm25)
X_pm10, y_pm10 = prepare_data(X_pm10, y_pm10)

# 모델 생성 및 학습
model_pm25 = LinearRegression()
model_pm25.fit(X_pm25, y_pm25)

model_pm10 = LinearRegression()
model_pm10.fit(X_pm10, y_pm10)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    ' pm25_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    ' pm10_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# PM2.5, PM10 예측
future_X_pm25 = future_beijing[[' pm25_beijing', ' pm10_beijing']]
future_X_pm10 = future_beijing[[' pm25_beijing', ' pm10_beijing']]

future_pred_pm25 = model_pm25.predict(future_X_pm25)
future_pred_pm10 = model_pm10.predict(future_X_pm10)

future_beijing['Predicted_PM25'] = future_pred_pm25
future_beijing['Predicted_PM10'] = future_pred_pm10

# AQI를 μg/m³로 변환하는 함수 정의 (한국 기준)
def aqi_conversion(pm_value, pm_type):
    if pm_type == 'PM2.5':
        if pm_value <= 12:
            return pm_value * (15.0 / 12)
        elif pm_value <= 35.4:
            return 15.0 + (pm_value - 12) * ((35.0 - 15.0) / (35.4 - 12))
        elif pm_value <= 75.4:
            return 35.0 + (pm_value - 35.4) * ((75.0 - 35.0) / (75.4 - 35.4))
        elif pm_value <= 115.4:
            return 75.0 + (pm_value - 75.4) * ((115.0 - 75.0) / (115.4 - 75.4))
        elif pm_value <= 150.4:
            return 115.0 + (pm_value - 115.4) * ((150.0 - 115.0) / (150.4 - 115.4))
        elif pm_value <= 500:
            return 150.0 + (pm_value - 150.4) * ((350.0 - 150.0) / (500 - 150.4))
        else:
            return 500.0
    elif pm_type == 'PM10':
        if pm_value <= 30:
            return pm_value * (30.0 / 30)
        elif pm_value <= 80:
            return 30.0 + (pm_value - 30) * ((80.0 - 30.0) / (80 - 30))
        elif pm_value <= 150:
            return 80.0 + (pm_value - 80) * ((150.0 - 80.0) / (150 - 80))
        elif pm_value <= 250:
            return 150.0 + (pm_value - 150) * ((250.0 - 150.0) / (250 - 150))
        elif pm_value <= 350:
            return 250.0 + (pm_value - 250) * ((350.0 - 250.0) / (350 - 250))
        elif pm_value <= 420:
            return 350.0 + (pm_value - 350) * ((420.0 - 350.0) / (420 - 350))
        else:
            return 420.0

# 예측 데이터를 AQI로 변환
future_beijing['Predicted_AQI_PM25'] = future_beijing['Predicted_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
future_beijing['Predicted_AQI_PM10'] = future_beijing['Predicted_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' pm25', ' pm10']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_PM25', 'Actual_PM10']

# 'Actual_PM25'와 'Actual_PM10' 열을 숫자로 변환
actual_seoul['Actual_PM25'] = pd.to_numeric(actual_seoul['Actual_PM25'], errors='coerce')
actual_seoul['Actual_PM10'] = pd.to_numeric(actual_seoul['Actual_PM10'], errors='coerce')

# 데이터를 AQI로 변환
actual_seoul['Actual_AQI_PM25'] = actual_seoul['Actual_PM25'].apply(lambda x: aqi_conversion(x, 'PM2.5'))
actual_seoul['Actual_AQI_PM10'] = actual_seoul['Actual_PM10'].apply(lambda x: aqi_conversion(x, 'PM10'))

# 오차 비율 계산
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')
combined_data['PM25_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM25'] - combined_data['Predicted_AQI_PM25']) / combined_data['Actual_AQI_PM25']) * 100
combined_data['PM10_Error_Rate'] = np.abs((combined_data['Actual_AQI_PM10'] - combined_data['Predicted_AQI_PM10']) / combined_data['Actual_AQI_PM10']) * 100

# 함수 정의: AQI 값을 입력받아 범주를 반환하는 함수
def categorize_aqi(aqi_value):
    if aqi_value <= 100:
        return 1
    elif aqi_value <= 200:
        return 2
    elif aqi_value <= 300:
        return 3
    else:
        return 4

# 예측된 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_predicted_aqi(df):
    df['Predicted_AQI_PM25_Category'] = df['Predicted_AQI_PM25'].apply(categorize_aqi)
    df['Predicted_AQI_PM10_Category'] = df['Predicted_AQI_PM10'].apply(categorize_aqi)
    return df

# 실제 AQI 값을 범주형으로 변환하는 함수 정의
def categorize_actual_aqi(df):
    df['Actual_AQI_PM25_Category'] = df['Actual_AQI_PM25'].apply(categorize_aqi)
    df['Actual_AQI_PM10_Category'] = df['Actual_AQI_PM10'].apply(categorize_aqi)
    return df

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['Predicted_AQI_PM25_Category'] != row['Actual_AQI_PM25_Category']:
        row['PM25_Category_Difference'] = 1
    else:
        row['PM25_Category_Difference'] = 0
    
    if row['Predicted_AQI_PM10_Category'] != row['Actual_AQI_PM10_Category']:
        row['PM10_Category_Difference'] = 1
    else:
        row['PM10_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_categorized = categorize_predicted_aqi(combined_data)
combined_data_categorized = categorize_actual_aqi(combined_data_categorized)

# combined_data_categorized에 함수 적용
combined_data_categorized_diff = combined_data_categorized.apply(compare_aqi_categories, axis=1)

# 결과 출력
print("2024년 미세먼지 예측 적중률")
# combined_data_categorized_diff[['date', 'Predicted_AQI_PM25_Category', 'Actual_AQI_PM25_Category', 
#                                       'PM25_Category_Difference', 'Predicted_AQI_PM10_Category', 
#                                       'Actual_AQI_PM10_Category', 'PM10_Category_Difference']]

# 2024년 한 해 동안의 데이터 필터링
total_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-12-31')]

year_data = total_data[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-06-30')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days = len(year_data)
same_pm25_category_count = year_data[year_data['PM25_Category_Difference'] == 0].shape[0]
same_pm10_category_count = year_data[year_data['PM10_Category_Difference'] == 0].shape[0]

percent_same_pm25_category = (same_pm25_category_count / total_days) * 100
percent_same_pm10_category = (same_pm10_category_count / total_days) * 100

# 결과 출력
print(f"2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: {percent_same_pm25_category:.2f}%")
print(f"2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: {percent_same_pm10_category:.2f}%\n")

# 특정 날짜의 데이터 출력 함수
def print_aqi_for_date(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data[combined_data['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
        print(f"예상 PM25 : {round(row['Predicted_AQI_PM25'].values[0])} (Category: {row['Predicted_AQI_PM25_Category'].values[0]})")
        print(f"예상 PM10 : {round(row['Predicted_AQI_PM10'].values[0])} (Category: {row['Predicted_AQI_PM10_Category'].values[0]})")

# 특정 날짜의 AQI 데이터를 출력
target_date = '2024-09-30'
print_aqi_for_date(target_date)

2024년 미세먼지 예측 적중률
2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: 79.67%
2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: 98.90%

2024-09-30 대기질 예측 데이터
예상 PM25 : 93 (Category: 1)
예상 PM10 : 52 (Category: 1)


In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    numeric_df.rename(columns={' pm25': 'co', ' pm10': 'o3'}, inplace=True)  # Rename columns
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# CO 예측 모델
X_co = corr_beijing_seoul[['co_beijing', 'o3_beijing']]
y_co = corr_beijing_seoul['co_seoul']

# O3 예측 모델
X_o3 = corr_beijing_seoul[['co_beijing', 'o3_beijing']]
y_o3 = corr_beijing_seoul['o3_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# CO, O3 데이터 준비
X_co, y_co = prepare_data(X_co, y_co)
X_o3, y_o3 = prepare_data(X_o3, y_o3)

# 모델 생성 및 학습
model_co = LinearRegression()
model_co.fit(X_co, y_co)

model_o3 = LinearRegression()
model_o3.fit(X_o3, y_o3)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    'co_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    'o3_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# CO, O3 예측
future_X_co = future_beijing[['co_beijing', 'o3_beijing']]
future_X_o3 = future_beijing[['co_beijing', 'o3_beijing']]

future_pred_co = model_co.predict(future_X_co)
future_pred_o3 = model_o3.predict(future_X_o3)

future_beijing['Predicted_CO'] = future_pred_co
future_beijing['Predicted_O3'] = future_pred_o3

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' pm25', ' pm10']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_CO', 'Actual_O3']

# 'Actual_CO'와 'Actual_O3' 열을 숫자로 변환
actual_seoul['Actual_CO'] = pd.to_numeric(actual_seoul['Actual_CO'], errors='coerce')
actual_seoul['Actual_O3'] = pd.to_numeric(actual_seoul['Actual_O3'], errors='coerce')

# 데이터 병합
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')

# O3 예측 오차 범주화 함수 정의
def categorize_o3_error(error):
    if error <= 100:
        return 1
    elif error <= 200:
        return 2
    elif error <= 300:
        return 3
    elif error <= 400:
        return 4
    else:
        return 0

combined_data['CO_Predict_Category'] = combined_data['Predicted_CO'].apply(categorize_o3_error)
combined_data['O3_Predict_Category'] = combined_data['Predicted_O3'].apply(categorize_o3_error)
combined_data['CO_Actual_Category'] = combined_data['Actual_CO'].apply(categorize_o3_error)
combined_data['O3_Actual_Category'] = combined_data['Actual_O3'].apply(categorize_o3_error)

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['CO_Predict_Category'] != row['CO_Actual_Category']:
        row['CO_Category_Difference'] = 1
    else:
        row['CO_Category_Difference'] = 0
    
    if row['O3_Predict_Category'] != row['O3_Actual_Category']:
        row['O3_Category_Difference'] = 1
    else:
        row['O3_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_diff = combined_data.apply(compare_aqi_categories, axis=1)

# 2024년 한 해 동안의 데이터 필터링
total_data = combined_data_categorized_diff[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-12-31')]

year_data = total_data[(combined_data_categorized_diff['date'] >= '2024-01-01') & 
                                           (combined_data_categorized_diff['date'] <= '2024-06-30')]

# PM2.5 및 PM10의 카테고리가 같은 비율 계산
total_days = len(year_data)
same_co_category_count = year_data[year_data['CO_Category_Difference'] == 0].shape[0]
same_o3_category_count = year_data[year_data['O3_Category_Difference'] == 0].shape[0]

percent_same_co_category = (same_co_category_count / total_days) * 100
percent_same_o3_category = (same_o3_category_count / total_days) * 100

# 결과 출력
print(f"2024년 동안 PM25의 예측 카테고리가 실제와 같은 비율: {percent_same_co_category:.2f}%")
print(f"2024년 동안 PM10의 예측 카테고리가 실제와 같은 비율: {percent_same_o3_category:.2f}%\n")

# 특정 날짜의 AQI 데이터를 출력 함수
def print_aqi_for_date(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data[combined_data['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
        print(f"예상 CO : {round(row['Predicted_CO'].values[0])} (Category: {row['CO_Predict_Category'].values[0]})")
        print(f"예상 O3 : {round(row['Predicted_O3'].values[0])} (Category: {row['O3_Predict_Category'].values[0]})")
        actual_co = round(row['Actual_CO'].values[0]) if not np.isnan(row['Actual_CO'].values[0]) else 'N/A'
        actual_o3 = round(row['Actual_O3'].values[0]) if not np.isnan(row['Actual_O3'].values[0]) else 'N/A'
        actual_co_category = row['CO_Actual_Category'].values[0] if not np.isnan(row['Actual_CO'].values[0]) else 'N/A'
        actual_o3_category = row['O3_Actual_Category'].values[0] if not np.isnan(row['Actual_O3'].values[0]) else 'N/A'
        print(f"실제 CO : {actual_co} (Category: {actual_co_category})")
        print(f"실제 O3 : {actual_o3} (Category: {actual_o3_category})")

# 특정 날짜의 AQI 데이터를 출력
target_date = '2024-12-25'
print_aqi_for_date(target_date)


In [36]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

# 데이터 불러오기 (경로는 적절히 변경해주세요)
beijing = pd.read_csv('D:\\FILE\\랴오청대기질.csv')
seoul = pd.read_csv('D:\\FILE\\서울대기질.csv')

# 날짜 열을 datetime 형식으로 변환
beijing['date'] = pd.to_datetime(beijing['date'], errors='coerce')
seoul['date'] = pd.to_datetime(seoul['date'], errors='coerce')

# 2023년 12월 31일까지의 데이터 필터링
beijing_past = beijing[beijing['date'] <= '2023-12-31']
seoul_past = seoul[seoul['date'] <= '2023-12-31']

# 숫자 데이터만 추출하고 비어있는 값을 NaN으로 대체하는 함수
def preprocess(df):
    numeric_df = df.drop(columns=['date']).apply(pd.to_numeric, errors='coerce')
    numeric_df.replace(' ', np.nan, inplace=True)
    numeric_df.fillna(numeric_df.mean(), inplace=True)
    # 컬럼명 소문자로 변환
    numeric_df.columns = map(str.lower, numeric_df.columns)
    numeric_df.rename(columns={' pm25': 'no2', ' pm10': 'so2'}, inplace=True)  # Rename columns
    return numeric_df

# 각 도시의 데이터프레임을 숫자로 변환하여 처리
beijing_numeric = preprocess(beijing_past)
seoul_numeric = preprocess(seoul_past)

# 서울과 베이징의 데이터 병합
corr_beijing_seoul = pd.merge(beijing_numeric, seoul_numeric, 
                              left_index=True, right_index=True, suffixes=('_beijing', '_seoul'))

# NO2 예측 모델
X_no2 = corr_beijing_seoul[['no2_beijing', 'so2_beijing']]
y_no2 = corr_beijing_seoul['no2_seoul']

# SO2 예측 모델
X_so2 = corr_beijing_seoul[['no2_beijing', 'so2_beijing']]
y_so2 = corr_beijing_seoul['so2_seoul']

# 데이터 준비 함수
def prepare_data(X, y):
    X = X.copy()
    y = y.copy()
    X.replace(' ', np.nan, inplace=True)
    X.fillna(X.astype(float).mean(), inplace=True)
    y.replace(' ', np.nan, inplace=True)
    y.fillna(y.astype(float).mean(), inplace=True)
    return X.astype(float), y.astype(float)

# NO2, SO2 데이터 준비
X_no2, y_no2 = prepare_data(X_no2, y_no2)
X_so2, y_so2 = prepare_data(X_so2, y_so2)

# 모델 생성 및 학습
model_no2 = LinearRegression()
model_no2.fit(X_no2, y_no2)

model_so2 = LinearRegression()
model_so2.fit(X_so2, y_so2)

# 2024년 1월 1일부터 12월 31일까지의 데이터 생성
future_dates = pd.date_range(start='2024-01-01', end='2024-12-31')
future_beijing = pd.DataFrame({
    'date': future_dates,
    'no2_beijing': [random.randint(50, 150) for _ in range(len(future_dates))],
    'so2_beijing': [random.randint(30, 100) for _ in range(len(future_dates))]
})

# NO2, SO2 예측
future_X_no2 = future_beijing[['no2_beijing', 'so2_beijing']]
future_X_so2 = future_beijing[['no2_beijing', 'so2_beijing']]

future_pred_no2 = model_no2.predict(future_X_no2)
future_pred_so2 = model_so2.predict(future_X_so2)

future_beijing['Predicted_NO2'] = future_pred_no2
future_beijing['Predicted_SO2'] = future_pred_so2

# 실제 데이터를 가져옴 (서울)
actual_seoul = seoul[(seoul['date'] >= '2024-01-01') & (seoul['date'] <= '2024-12-31')][['date', ' pm25', ' pm10']].reset_index(drop=True)
actual_seoul.columns = ['date', 'Actual_NO2', 'Actual_SO2']

# 'Actual_NO2'와 'Actual_SO2' 열을 숫자로 변환
actual_seoul['Actual_NO2'] = pd.to_numeric(actual_seoul['Actual_NO2'], errors='coerce')
actual_seoul['Actual_SO2'] = pd.to_numeric(actual_seoul['Actual_SO2'], errors='coerce')

# 데이터 병합
combined_data = pd.merge(future_beijing, actual_seoul, on='date', how='left')

# SO2 예측 오차 범주화 함수 정의
def categorize_so2_error(error):
    if error <= 100:
        return 1
    elif error <= 200:
        return 2
    elif error <= 300:
        return 3
    elif error <= 400:
        return 4
    else:
        return 0

combined_data['NO2_Predict_Category'] = combined_data['Predicted_NO2'].apply(categorize_so2_error)
combined_data['SO2_Predict_Category'] = combined_data['Predicted_SO2'].apply(categorize_so2_error)
combined_data['NO2_Actual_Category'] = combined_data['Actual_NO2'].apply(categorize_so2_error)
combined_data['SO2_Actual_Category'] = combined_data['Actual_SO2'].apply(categorize_so2_error)

# 예측된 AQI 카테고리와 실제 AQI 카테고리를 비교하여 다른 경우 1, 같은 경우 0을 반환
def compare_aqi_categories(row):
    if row['NO2_Predict_Category'] != row['NO2_Actual_Category']:
        row['NO2_Category_Difference'] = 1
    else:
        row['NO2_Category_Difference'] = 0
    
    if row['SO2_Predict_Category'] != row['SO2_Actual_Category']:
        row['SO2_Category_Difference'] = 1
    else:
        row['SO2_Category_Difference'] = 0
    
    return row

# COMBINED_DATA에 적용
combined_data_diff = combined_data.apply(compare_aqi_categories, axis=1)

# 2024년 한 해 동안의 데이터 필터링
total_data = combined_data_diff[(combined_data_diff['date'] >= '2024-01-01') & 
                                (combined_data_diff['date'] <= '2024-12-31')]

year_data = total_data[(total_data['date'] >= '2024-01-01') & 
                       (total_data['date'] <= '2024-06-30')]

# NO2 및 SO2의 카테고리가 같은 비율 계산
total_days = len(year_data)
same_no2_category_count = year_data[year_data['NO2_Category_Difference'] == 0].shape[0]
same_so2_category_count = year_data[year_data['SO2_Category_Difference'] == 0].shape[0]

percent_same_no2_category = (same_no2_category_count / total_days) * 100
percent_same_so2_category = (same_so2_category_count / total_days) * 100

# 결과 출력
print(f"2024년 동안 NO2의 예측 카테고리가 실제와 같은 비율: {percent_same_no2_category:.2f}%")
print(f"2024년 동안 SO2의 예측 카테고리가 실제와 같은 비율: {percent_same_so2_category:.2f}%\n")

# 특정 날짜의 AQI 데이터를 출력 함수
def print_aqi_for_date(target_date):
    target_date = pd.to_datetime(target_date)
    row = combined_data[combined_data['date'] == target_date]
    if row.empty:
        print(f"No data available for the date: {target_date.strftime('%Y-%m-%d')}")
    else:
        print(f"{target_date.strftime('%Y-%m-%d')} 대기질 예측 데이터")
        print(f"예상 NO2 : {round(row['Predicted_NO2'].values[0])} (Category: {row['NO2_Predict_Category'].values[0]})")
        print(f"예상 SO2 : {round(row['Predicted_SO2'].values[0])} (Category: {row['SO2_Predict_Category'].values[0]})")
        actual_no2 = round(row['Actual_NO2'].values[0]) if not np.isnan(row['Actual_NO2'].values[0]) else 'N/A'
        actual_so2 = round(row['Actual_SO2'].values[0]) if not np.isnan(row['Actual_SO2'].values[0]) else 'N/A'
        actual_no2_category = row['NO2_Actual_Category'].values[0] if not np.isnan(row['Actual_NO2'].values[0]) else 'N/A'
        actual_so2_category = row['SO2_Actual_Category'].values[0] if not np.isnan(row['Actual_SO2'].values[0]) else 'N/A'
        print(f"실제 NO2 : {actual_no2} (Category: {actual_no2_category})")
        print(f"실제 SO2 : {actual_so2} (Category: {actual_so2_category})")

# 특정 날짜의 AQI 데이터를 출력
target_date = '2024-12-25'
print_aqi_for_date(target_date)


2024년 동안 NO2의 예측 카테고리가 실제와 같은 비율: 79.67%
2024년 동안 SO2의 예측 카테고리가 실제와 같은 비율: 98.90%

2024-12-25 대기질 예측 데이터
예상 NO2 : 79 (Category: 1)
예상 SO2 : 43 (Category: 1)
실제 NO2 : N/A (Category: N/A)
실제 SO2 : N/A (Category: N/A)
